In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dataset_name = 'cats_vs_dogs'
(train_dataset, validation_dataset), info = tfds.load(name=dataset_name, split=('train[:80%]', 'train[-20%:]'), with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\shoni\tensorflow_datasets\cats_vs_dogs\4.0.0.incompleteAPBAVM\cats_vs_dogs-train.tfrecord*.…

Dataset cats_vs_dogs downloaded and prepared to C:\Users\shoni\tensorflow_datasets\cats_vs_dogs\4.0.0. Subsequent calls will reuse this data.


In [3]:
len(train_dataset)

18610

In [4]:
len(validation_dataset)

4652

In [5]:
list(train_dataset)[0]

{'image': <tf.Tensor: shape=(262, 350, 3), dtype=uint8, numpy=
 array([[[242, 248, 248],
         [240, 246, 246],
         [235, 239, 238],
         ...,
         [188, 174, 127],
         [145, 133,  85],
         [161, 149, 101]],
 
        [[238, 244, 244],
         [239, 245, 245],
         [235, 239, 238],
         ...,
         [188, 176, 128],
         [165, 153, 105],
         [178, 168, 119]],
 
        [[237, 241, 240],
         [238, 242, 241],
         [232, 236, 235],
         ...,
         [200, 187, 142],
         [201, 191, 142],
         [200, 192, 143]],
 
        ...,
 
        [[130, 122,  75],
         [152, 146,  98],
         [154, 148,  98],
         ...,
         [161, 166, 136],
         [ 77,  86,  57],
         [ 71,  82,  52]],
 
        [[167, 159, 112],
         [105,  99,  51],
         [148, 142,  92],
         ...,
         [ 67,  73,  45],
         [ 46,  56,  29],
         [ 41,  54,  26]],
 
        [[138, 130,  83],
         [ 83,  77,  29],
     

In [6]:
def normalize(images):
    # 0~1 사이의 값으로 Normalize 합니다.
    img, lbl = tf.cast(images['image'], tf.float32) / 255.0, images['label']
    # 300 X 300 사이즈로 resize 합니다.
    img = tf.image.resize(img, size=(300, 300))
    return img, lbl

In [7]:
BATCH_SIZE = 64
epochs=20

In [8]:
train_data = train_dataset.map(normalize).batch(BATCH_SIZE).repeat().prefetch(1)
valid_data = validation_dataset.map(normalize).batch(BATCH_SIZE).repeat().prefetch(1)
steps_per_epoch= int(len(list(train_dataset)) * 0.8) // BATCH_SIZE + 1
validation_steps= int(len(list(validation_dataset)) * 0.2) // BATCH_SIZE + 1

In [9]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

58889256/58889256 [==============================] - 3s 0us/step


In [10]:
transfer_model.trainable=False

In [11]:
model_tr = Sequential([
    transfer_model,
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, 'sigmoid')
])

In [12]:
model_tr.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
import datetime
log_dir = "/logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [14]:
transfer_history = model_tr.fit(train_data,
                             validation_data=(valid_data),
                             steps_per_epoch=steps_per_epoch,
                             validation_steps=validation_steps,
                             epochs=20, callbacks=[tensorboard_callback]
                            )

Epoch 1/20
233/233 [==============================] - 4332s 19s/step - loss: 0.3122 - accuracy: 0.8827 - val_loss: 0.1510 - val_accuracy: 0.9490
Epoch 2/20
202/233 [=========================>....] - ETA: 11:32 - loss: 0.1494 - accuracy: 0.9418

KeyboardInterrupt: 